# **dim_customer table gold**

Start by cereting the gold schema and adding the comment

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bikes.gold
COMMENT 'Gold layer schema containing curated dimension and fact tables for analytics';

Add Tags for the Schema

In [0]:
%sql
ALTER SCHEMA bikes.gold
SET TAGS (
  'layer' = 'gold',
  'domain' = 'sales',
  'owner' = 'data_engineering_team'
);

Creating the dim_customer table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bikes.gold.dim_customers (
  customer_id INT,
  customer_key STRING,
  first_name STRING,
  last_name STRING,
  marital_status STRING,
  gender STRING,
  date_of_birth DATE,
  country STRING,
  created_date DATE
)
COMMENT 'Customer dimension table containing customer details';

Adding comments to columns

In [0]:
%sql
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN customer_id COMMENT 'Surrogate customer identifier from CRM';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN customer_key COMMENT 'Business customer key used across systems';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN first_name COMMENT 'Customer first name';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN last_name COMMENT 'Customer last name';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN marital_status COMMENT 'Customer marital status (Single / Married)';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN gender COMMENT 'Customer gender';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN date_of_birth COMMENT 'Customer date of birth from ERP system';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN country COMMENT 'Customer country from ERP location system';
ALTER TABLE bikes.gold.dim_customers ALTER COLUMN created_date COMMENT 'Customer record creation date in CRM';

Inserting the data by aggregation all costomers tables form CRM and ERP

In [0]:
%sql
CREATE OR REPLACE TABLE bikes.gold.dim_customers AS
SELECT 
    c.customer_id,
    c.customer_key,
    c.first_name,
    c.last_name,
    c.marital_status,
    c.gender,
    ec.date_of_birth,
    loc.country,
    c.created_date
FROM bikes.silver.crm_customer_info AS c
LEFT JOIN bikes.silver.erp_customer AS ec
    ON concat('NASAW000', c.customer_id) = ec.customer_id
LEFT JOIN bikes.silver.erp_loc loc
    ON regexp_replace(loc.customer_id, '-', '') = c.customer_key;

Adding tags to the table

In [0]:
%sql
ALTER TABLE bikes.gold.dim_customers
SET TAGS (
  'layer' = 'gold',
  'table_type' = 'dimension',
  'domain' = 'sales',
  'pii' = 'true'
);


ALTER TABLE bikes.gold.dim_customers
ALTER COLUMN first_name
SET TAGS ('pii' = 'true');

ALTER TABLE bikes.gold.dim_customers
ALTER COLUMN last_name
SET TAGS ('pii' = 'true');

ALTER TABLE bikes.gold.dim_customers
ALTER COLUMN date_of_birth
SET TAGS ('pii' = 'true');

ALTER TABLE bikes.gold.dim_customers
ALTER COLUMN customer_id
SET TAGS ('pii' = 'false');